### Demonstration Overview: Translating a VRS (version 2.0) object into an Allele Profile

This notebook demonstrates the translation of GA4GH VRS (version 2.0) into the HL7 FHIR Allele format. 

In this notebook, we use example data to generate VRS objects with the `vrs-python` library. Once created, these objects can be converted into FHIR-compliant Allele using the `vrs_allele_to_allele_profile` method from the `VrsFhirAlleleTranslation` class. This method enables a one-way transformation from VRS to FHIR Allele.

### Prerequisites and Setup

To support the one way transformation from VRS Allele to FHIR Alelle of this demonstration, we set up the environment by importing the necessary libraries and modules. These include:

1. **External Package**:
   - `models` from `ga4gh.vrs`: Provides foundational data models for working with GA4GH Variation Representation Schema (VRS).

2. **Custom Project Modules**:
   - `AlleleNormalizer` from `normalize.allele_normalizer`: A utility for normalizing a vrs allele object.
   - `VrsFhirAlleleTranslation` from `translators.allele_translator`: A translation component for converting VRS alleles into FHIR Allele.

In [ ]:
# importing the vrs models
from ga4gh.vrs.models import SequenceLocation,SequenceReference,LiteralSequenceExpression,sequenceString,Allele
from translators.allele_translator import VrsFhirAlleleTranslation
from normalizers.allele_normalizer import AlleleNormalizer
from api.seqrepo import SeqRepoAPI


normalize = AlleleNormalizer()
alleleTranslator = VrsFhirAlleleTranslation()
seqrepo_api = SeqRepoAPI()

### Example 1: Creating, Normalizing, and Translating a VRS Allele

In this example, we first generate a VRS allele object and normalize it. Next, we translate the normalized allele into a FHIR-compliant Allele.

In [ ]:
# Creating the VRS object and then normalizing it.
# Example 1 - Deletion origin: "NC_000001.11:g.1014265del"
start = 1014263
end = 1014265
refseq = "NC_000001.11"
alt_seq = "C"


refget_accession = seqrepo_api.seqrepo_dataproxy.derive_refget_accession(f"refseq:{refseq}")
seq_ref = SequenceReference(
    refgetAccession=refget_accession.split("refget:")[-1]
    )

seq_location = SequenceLocation(
    sequenceReference=seq_ref,
    start = start,
    end=end,
)
lit_seq_expr = LiteralSequenceExpression(
    sequence=sequenceString(alt_seq)
)
del_example_1 = Allele(
    location=seq_location,
    state=lit_seq_expr
)

norm_del_example_1 = normalize.post_normalize_allele(del_example_1)
norm_del_example_1.model_dump(exclude_none=True)
#TODO: this example is a ReferenceLengthExpression: It will not translate because we only translate literalsequenceexpression. Need to find a work around. 

In [ ]:
from ga4gh.vrs.normalize import denormalize_reference_length_expression
#TODO: come back to this
# denormalize_reference_length_expression(ref_seq="c",repeat_subunit_length=1,alt_length=2)
ref_seq = "c"
repeat_subunit_length = norm_del_example_1.state.repeatSubunitLength
alt_length = norm_del_example_1.location.end - norm_del_example_1.location.start

denormalize_reference_length_expression(ref_seq, repeat_subunit_length, alt_length)
# norm_del_example_1.location.end - norm_del_example_1.location.start

In [ ]:
# Translating the normalized allele into an allele profile
allele_profile_del_example = alleleTranslator.vrs_allele_to_allele_profile(norm_del_example_1)

print(type(allele_profile_del_example))
allele_profile_del_example.model_dump()

### Example 2 

In [ ]:
# Creating the VRS object and then normalizing it.
# Example 2 - Insertion origin: "NC_000001.11:g.113901365_113901366insATA"
start = 113901365
end = 113901365
refseq = "NC_000001.11"
alt_seq = "ATA"


refget_accession = seqrepo_api.seqrepo_dataproxy.derive_refget_accession(f"refseq:{refseq}")
seq_ref = SequenceReference(
    refgetAccession=refget_accession.split("refget:")[-1]
    )

seq_location = SequenceLocation(
    sequenceReference=seq_ref,
    start = start,
    end=end,
)
lit_seq_expr = LiteralSequenceExpression(
    sequence=sequenceString(alt_seq)
)
insertion_example = Allele(
    location=seq_location,
    state=lit_seq_expr
)

norm_insertion_example = normalize.post_normalize_allele(insertion_example)
norm_insertion_example.model_dump(exclude_none=True)

In [ ]:
# Translating the normalized allele into an allele profile
allele_profile_insertion = alleleTranslator.vrs_allele_to_allele_profile(norm_insertion_example)

print(type(allele_profile_insertion))
allele_profile_insertion.model_dump()

### Example 3

In [ ]:
# Creating the VRS object and then normalizing it.
# Example 3 - Substituion origin: "NC_000002.12:g.27453449C>T"
start = 27453448
end = 27453449
refseq = "NC_000002.12"
alt_seq = "T"


refget_accession = seqrepo_api.seqrepo_dataproxy.derive_refget_accession(f"refseq:{refseq}")
seq_ref = SequenceReference(
    refgetAccession=refget_accession.split("refget:")[-1]
    )

seq_location = SequenceLocation(
    sequenceReference=seq_ref,
    start = start,
    end=end,
)
lit_seq_expr = LiteralSequenceExpression(
    sequence=sequenceString(alt_seq)
)
sub_example = Allele(
    location=seq_location,
    state=lit_seq_expr
)

norm_sub_example = normalize.post_normalize_allele(sub_example)
norm_sub_example.model_dump(exclude_none=True)

In [ ]:
# Translating the normalized allele into an allele profile
allele_profile_sub_example = alleleTranslator.vrs_allele_to_allele_profile(norm_sub_example)

print(type(allele_profile_sub_example))
allele_profile_sub_example.model_dump()

### Conclusion

In this notebook, we demonstrated the translation of GA4GH VRS alleles into HL7 FHIR Allele. We began by creating VRS allele objects, normalizing them, and converting them into FHIR-compliant representations using the VrsFhirAlleleTranslation class. This process enhances interoperability between GA4GH and HL7 FHIR standards, enabling seamless integration of genomic data into healthcare systems. The examples included deletion, insertion, and substitution variants.

We recognize that VRS is continuously evolving, with VRS 2.0 on the horizon. As the new version stabilizes, we plan to assess its impact and potentially refactor our implementation to align with the latest standard, ensuring continued compatibility and functionality.